### LLM Binding Tools

In [ ]:
from dotenv import load_dotenv

load_dotenv()

In [1]:
from langchain_teddynote import logging

logging.langsmith("edu-CH15-Bind-Tools")

LangSmith 추적을 시작합니다.
[프로젝트명]
edu-CH15-Bind-Tools


### LLM에 바인딩할 Tool 정의 

In [2]:
import re 
import requests
from bs4 import BeautifulSoup
from langchain.agents import tool

# 도구 정의 
@tool 
def get_word_length(word: str) -> int:
    """Returns the length of a word."""
    return len(word)

@tool 
def add_function(a: float, b: float) -> float:
    """Adds two numbers together."""
    return a + b

@tool 
def naver_new_crawl(news_url: str) -> str:
    """Crawls a 네이버 (naver.com) news article and returns the body content."""
    response = requests.get(news_url)
    
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, "html.parser")
        
        title = soup.find("h2", id="title_area").get_text()
        content = soup.find("div", id="contents").get_text()
        cleaned_title = re.sub(r"\n{2,}", "\n", title)
        cleaned_content = re.sub(r"\n{2,}", "\n", content)
    else:
        print(f"HTTP 요청 실패. 응답 코드: {response.status_code}")

tools = [get_word_length, add_function, naver_new_crawl]

### bind_tools()로 LLM에 도구 바인딩 

In [17]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

llm_with_tools = llm.bind_tools(tools)

In [18]:
question = "What is the length of the word 'teddynote'?"
llm_with_tools.invoke(question).tool_calls

[{'name': 'get_word_length',
  'args': {'word': 'teddynote'},
  'id': 'call_PPrWIABQTsnEWLnV2c2ATpdc',
  'type': 'tool_call'}]

In [19]:
from langchain_core.output_parsers.openai_tools import JsonOutputToolsParser

# 도구 바인디 + 도구 파서 
chain = llm_with_tools | JsonOutputToolsParser(tool=tools)

tool_call_results = chain.invoke(question)

In [20]:
print(tool_call_results)

[{'args': {'word': 'teddynote'}, 'type': 'get_word_length'}]


In [21]:
print(tool_call_result, end="\n\n==========\n\n")
single_result = tool_call_results[0]
# 도구 이름
print(single_result["type"])
# 도구 인자
print(single_result["args"])

[{'args': {'word': 'teddynote'}, 'type': 'get_word_length'}]


get_word_length
{'word': 'teddynote'}


In [22]:
tools

[StructuredTool(name='get_word_length', description='Returns the length of a word.', args_schema=<class 'langchain_core.utils.pydantic.get_word_length'>, func=<function get_word_length at 0x00000154B22C7560>),
 StructuredTool(name='add_function', description='Adds two numbers together.', args_schema=<class 'langchain_core.utils.pydantic.add_function'>, func=<function add_function at 0x00000154B228F060>),
 StructuredTool(name='naver_new_crawl', description='Crawls a 네이버 (naver.com) news article and returns the body content.', args_schema=<class 'langchain_core.utils.pydantic.naver_new_crawl'>, func=<function naver_new_crawl at 0x00000154B22C6DE0>)]

In [23]:
tool_call_results[0]["type"], tools[0].name

('get_word_length', 'get_word_length')

In [25]:
def execute_tool_calls(tool_call_results):
    """
    도구 호출 결과를 실행하는 함수

    :param tool_call_results: 도구 호출 결과 리스트
    :param tools: 사용 가능한 도구 리스트
    """
    
    for tool_call_result in tool_call_results:
        tool_name = tool_call_result["type"]  # 도구의 이름(함수명)
        tool_args = tool_call_result["args"]  # 도구에 전달되는 인자

        matching_tool = next((tool for tool in tools if tool.name == tool_name), None)

        if matching_tool:
            result = matching_tool.invoke(tool_args)
            print(f"[실행도구] {tool_name} [Argument] {tool_args}\n[실행결과] {result}")
        else:
            print(f"경고: {tool_name}에 해당하는 도구를 찾을 수 없습니다.")

execute_tool_calls(tool_call_results)

[실행도구] get_word_length [Argument] {'word': 'teddynote'}
[실행결과] 9


### bind_tools + Parser + Execution

In [26]:
from langchain_core.output_parsers.openai_tools import JsonOutputToolsParser

chain = llm_with_tools | JsonOutputToolsParser(tools=tools) | execute_tool_calls

In [27]:
chain.invoke(question)

[실행도구] get_word_length [Argument] {'word': 'teddynote'}
[실행결과] 9


In [28]:
chain.invoke("114.5 + 121.2")

[실행도구] add_function [Argument] {'a': 114.5, 'b': 121.2}
[실행결과] 235.7


In [29]:
chain.invoke(
    "뉴스 기사 내용을 크롤링해줘: https://n.news.naver.com/mnews/hotissue/article/092/0002347672?type=series&cid=2000065"
)

[실행도구] naver_new_crawl [Argument] {'news_url': 'https://n.news.naver.com/mnews/hotissue/article/092/0002347672?type=series&cid=2000065'}
[실행결과] None


### bind_tools > Agent & AgentExecutor로 대체

In [30]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are very powerful assistant, but don't know current events",),
        ("human", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad")
    ]
)

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

In [31]:
from langchain.agents import create_tool_calling_agent
from langchain.agents import AgentExecutor

tools = [get_word_length, add_function, naver_new_crawl]

agent = create_tool_calling_agent(llm, tools, prompt)

agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True
)

In [32]:
result = agent_executor.invoke({"input": question})
print(result["output"])



> Entering new AgentExecutor chain...

Invoking: `get_word_length` with `{'word': 'teddynote'}`


9The length of the word 'teddynote' is 9.

> Finished chain.
The length of the word 'teddynote' is 9.


In [33]:
result = agent_executor.invoke({"input": "114.5 + 121.2 의 계산 결과는?"})

# 결과 확인
print(result["output"])



> Entering new AgentExecutor chain...

Invoking: `add_function` with `{'a': 114.5, 'b': 121.2}`


235.7114.5 + 121.2의 계산 결과는 235.7입니다.

> Finished chain.
114.5 + 121.2의 계산 결과는 235.7입니다.


In [34]:
result = agent_executor.invoke(
    {"input": "114.5 + 121.2 + 34.2 + 110.1 의 계산 결과는?"}
)

# 결과 확인
print(result["output"])
print("==========\n")
print(114.5 + 121.2 + 34.2 + 110.1)



> Entering new AgentExecutor chain...

Invoking: `add_function` with `{'a': 114.5, 'b': 121.2}`


235.7
Invoking: `add_function` with `{'a': 34.2, 'b': 110.1}`


144.3
Invoking: `add_function` with `{'a': 235.7, 'b': 144.3}`


380.0114.5 + 121.2 + 34.2 + 110.1의 계산 결과는 380.0입니다.

> Finished chain.
114.5 + 121.2 + 34.2 + 110.1의 계산 결과는 380.0입니다.

380.0


In [35]:
result = agent_executor.invoke(
    {
        "input": "뉴스 기사를 요약해 줘: https://n.news.naver.com/mnews/hotissue/article/092/0002347672?type=series&cid=2000065"
    }
)
print(result["output"])



> Entering new AgentExecutor chain...

Invoking: `naver_new_crawl` with `{'news_url': 'https://n.news.naver.com/mnews/hotissue/article/092/0002347672?type=series&cid=2000065'}`


None뉴스 기사를 가져오는 데 실패했습니다. 다른 방법으로 도와드릴까요?

> Finished chain.
뉴스 기사를 가져오는 데 실패했습니다. 다른 방법으로 도와드릴까요?
